In [2]:
!pip install -q torch torchvision transformers

In [11]:
# Install necessary libraries
!pip install -q torch torchvision transformers

# Import necessary libraries
import torch
import torchvision.transforms as T
from PIL import Image
from transformers import ViTFeatureExtractor, ViTForImageClassification, AutoTokenizer, AutoModelForSeq2SeqLM

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
# Initialize ViT model and feature extractor
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224").to(device)
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

# Load language generation model and tokenizer (e.g., T5 for text generation)
text_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small").to(device)
tokenizer = AutoTokenizer.from_pretrained("t5-small")

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Function to preprocess and transform the image for ViT
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=image, return_tensors="pt").to(device)
    return inputs

# Function to generate insights using embeddings from ViT
def generate_insights(image_path, question="Analyze this image"):
    # Preprocess image
    inputs = preprocess_image(image_path)
    
    # Extract embeddings using ViT
    with torch.no_grad():
        outputs = vit_model(**inputs)
    # Use pooled output as image representation for language model
    image_embedding = outputs.pooler_output

    # Generate insights by formatting question and image embeddings
    prompt = f"{question}. Image Embedding: {image_embedding.cpu().numpy().tolist()}"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # Generate response with text model
    with torch.no_grad():
        output = text_model.generate(input_ids, max_length=100)
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [18]:
# Example usage
image_path = "/mnt/code/test_l1.png"
question = "Analyze this graph"
response = generate_insights(image_path, question)

print(f"User: {question}")
print(f"Assistant: {response}")

Token indices sequence length is longer than the specified maximum sequence length for this model (7887 > 512). Running this sequence through the model will result in indexing errors


User: Analyze this graph
Assistant: -0.3272727878787, -0.327272727, -0.327272727, -0.327272727, -0.327272727, -0.327272727, -0.327272727, -0.32727272727, -0.32727272727, -0.32727272727, -0.327272727, -0.32727
